In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import pickle

/usr/local/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
# generate BeautifulSoup object 
url = "https://fairygodboss.com/maternity-leave-resource-center"
response = requests.post(url)
page = response.text
soup = BeautifulSoup(page,"html.parser")

In [3]:
# scrape company information
collect_cos = []
for line in soup.findAll('a',attrs = {'class':'comp_page'}):
    
    try:
        #filter out line breaks and remove extra white space
        co = [i.text.strip() for i in line.children if str(i) not in ['\n']] 
        
        #append company to list
        collect_cos.append(co)
        
    except:
        print "something's wrong"
        continue
        
print "Number of companies scraped: {0}".format(len(collect_cos))

Number of companies scraped: 945


In [4]:
# create pandas dataframe object
df = pd.DataFrame(collect_cos, columns = ['company','industry','paid','unpaid'])

df.head()

,company,industry,paid,unpaid
0,Bill and Melinda Gates Foundation,Nonprofit,52 weeks,N/A
1,Netflix,Technology: Consumer Internet,52 weeks,2 weeks
2,"Automattic, Inc.",Technology: Consumer Internet,32 weeks,0 weeks
3,Adobe Systems,Technology: Software,26 weeks,6 weeks
4,Motorola Mobility,Technology: Manufacturing,24 weeks,N/A


In [5]:
# Question 1: how many different industries are we looking at?

print "Industries: {0}".format(len(df.industry.unique()))

Industries: 127


In [6]:
# Question 2: are paid/unpaid time measured in units besides weeks?

for i,row in df.iterrows():
    if row['paid']!="N/A":
        if "weeks" not in row['paid']:
            print "Measured in units that are not weeks!"
    if row['unpaid']!="N/A":
        if "weeks" not in row['unpaid']:
            print "Measured in units that are not weeks!"

In [7]:
# Problem 1: convert paid and unpaid to numeric values

def extract_num(val):
    if val == "N/A":
        return None
    else:
        return int(val[:-6])

df['paid_leave'] = df['paid'].map(lambda x: extract_num(x))
df['unpaid_leave'] = df['unpaid'].map(lambda x: extract_num(x))

In [8]:
# Problem 2: clean industry field
# TODO: read in all industries as lower case

def clean_industry(val):
    if val == "N/A": #create 'Unspecified' value to capture all null values
        return "Unspecified"
    
    if val == "akqaa": #company AKQA is coded with industry akqaa. Convert to "Technology"
        return "Technology"
    
    if val == "Auto parts manufacturing" or val == "Auto parts": #combine Auto parts and Auto parts manufacturing
        return "Auto manufacturers"
    
    if val == "Consulting Services":
        return "Consulting services"
    
    else: #reduce number of industries by merging to base industry. eg. Technology: Software -> Technology
        return val.split(':')[0]
    
df['industry_parsed'] = df['industry'].map(lambda x: clean_industry(x))

In [9]:
# Question 1 (revisited): how many different industry types do we have?
# Industries inputted as "N/A" are now part of "Unspecified" category

print "Industries (parsed): {0}".format(len(df.industry_parsed.unique()))

Industries (parsed): 43


In [10]:
# Industry categories + counts

pd.DataFrame(df.industry_parsed.value_counts(dropna=False)).to_csv('./data/industry_counts_total.csv') #write to csv for chart

df.industry_parsed.value_counts(dropna=False)

Technology                      151
Legal services                   78
Retail                           73
Finance                          66
Educational Services             59
Healthcare                       55
Natural Resources                40
FMCG                             38
Insurance                        37
Media                            33
Nonprofit                        28
Industrial                       27
Consulting services              25
Pharmaceutical                   25
Hospitality                      21
Government                       19
Business Services                19
Advertising                      18
Information Services             18
Transportation                   17
Telecommunications               12
Auto manufacturers               10
Aerospace                         9
Unspecified                       9
Real Estate Company               7
Services                          7
Accounting services               6
Publishing                  

In [11]:
df.sort_values(by = 'paid_leave', ascending=False).head(10)

,company,industry,paid,unpaid,paid_leave,unpaid_leave,industry_parsed
0,Bill and Melinda Gates Foundation,Nonprofit,52 weeks,N/A,52,NaN,Nonprofit
1,Netflix,Technology: Consumer Internet,52 weeks,2 weeks,52,2,Technology
2,"Automattic, Inc.",Technology: Consumer Internet,32 weeks,0 weeks,32,0,Technology
3,Adobe Systems,Technology: Software,26 weeks,6 weeks,26,6,Technology
4,Motorola Mobility,Technology: Manufacturing,24 weeks,N/A,24,NaN,Technology
5,Spotify,Technology: Consumer Internet,24 weeks,N/A,24,NaN,Technology
6,FireEye Inc.,Technology: Security,22 weeks,N/A,22,NaN,Technology
7,"Orrick, Herrington & Sutcliffe LLP",Legal services,22 weeks,N/A,22,NaN,Legal services
8,Avaya,Technology: B2B Tech Services,22 weeks,N/A,22,NaN,Technology
11,Microsoft,Technology: Software,20 weeks,0 weeks,20,0,Technology


In [12]:
# size of dataset (number of companies)
print len(df)
print len(df.dropna())

945
469


In [13]:
print "Average paid leave: {0:.1f} weeks".format(df['paid_leave'].mean())
print "Average unpaid leave {0:.1f} weeks".format(df['unpaid_leave'].mean())

Average paid leave: 8.1 weeks
Average unpaid leave 8.6 weeks


In [14]:
#save number of companies, industries, and avg paid/unpaid leave to pickle file
#for chart and blogpost

d = {'num_co':len(df),
     'num_ind':len(df['industry_parsed'].unique()),
     'avg_paid':df['paid_leave'].mean(),
    'avg_unpaid':df['unpaid_leave'].mean()}

with open('./data/key_metrics.pkl','w') as pf:
    pickle.dump(d,pf)

In [15]:
#unspecified companies need to separated into corresponding industries
df[df['industry_parsed']=='Unspecified']

,company,industry,paid,unpaid,paid_leave,unpaid_leave,industry_parsed
122,Synergis,N/A,12 weeks,N/A,12,NaN,Unspecified
277,Cornerstone Chemical Company,N/A,10 weeks,2 weeks,10,2,Unspecified
704,QVC,N/A,2 weeks,0 weeks,2,0,Unspecified
737,Colorado Public Radio,N/A,0 weeks,N/A,0,NaN,Unspecified
738,Onward Search,N/A,0 weeks,N/A,0,NaN,Unspecified
739,Jenny Craig Inc.,N/A,0 weeks,N/A,0,NaN,Unspecified
740,AlliedBarton Security Services,N/A,0 weeks,4 weeks,0,4,Unspecified
741,SEPTA,N/A,0 weeks,12 weeks,0,12,Unspecified
810,menards,N/A,0 weeks,N/A,0,NaN,Unspecified


####WRITE TO CSV FILES

In [16]:
# group by industry

gb = df.groupby('industry_parsed')

# export average paid/unpaid leave (in weeks) to csv

gb['paid_leave','unpaid_leave'].mean().to_csv("./data/industry_means.csv")

In [17]:
# include count (number of companies inputted for each industry value)

df_temp = gb['paid_leave','unpaid_leave'].agg(['mean', 'count'])

# shape it!

df_temp = df_temp.stack(1).reset_index()
df_temp.columns = ['industry','measure','paid_leave','unpaid_leave']
df_temp = df_temp.set_index('industry')

df_temp.to_csv("./data/industry_means_counts.csv")

In [18]:
# reshaping to wide format 

df_temp_index = df_temp.reset_index()

# extend 'measure' column for both paid and unpaid values

unpaid = df_temp_index.pivot(index='industry',columns='measure',values='unpaid_leave').reset_index()
paid = df_temp_index.pivot(index='industry',columns='measure',values='paid_leave').reset_index()

wide = paid.merge(unpaid, on='industry', how = 'outer', suffixes = ["_paid","_unpaid"])
wide.set_index('industry')
wide.to_csv('./data/industries_wide.csv')

In [19]:
# reshaping it again, experiments!

trying = gb['paid_leave','unpaid_leave'].agg(['mean', 'count']).stack().reset_index()

trying_again = trying.pivot_table(values=['paid_leave','unpaid_leave'], index='industry_parsed', columns='level_1')
trying_again_no_index = trying_again.reset_index()

yes=pd.melt(trying_again_no_index, id_vars=['industry_parsed'],value_vars=['paid_leave','unpaid_leave'])
yes.columns = ['industry','type_leave','measure','val']

yes = yes.pivot_table(index=['industry','type_leave'],values='val', columns = 'measure')
yes.reset_index().to_csv("./data/industry_melted_v2.csv")